In [7]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

X = torch.rand(2, 20)
net(X)

tensor([[-0.0624,  0.0264, -0.0831,  0.2373,  0.0422, -0.0802, -0.0296,  0.1107,
          0.1394, -0.1504],
        [-0.1806, -0.0898, -0.1200,  0.2021, -0.0705,  0.0979, -0.0046,  0.0931,
         -0.0101, -0.1210]], grad_fn=<AddmmBackward0>)

In [22]:
# 自定义块
class MLP(nn.Module):
    # 用模型参数声明层
    def __init__(self):
        # 执行必要的初始化
        super().__init__() # 调用父类初始化（__init__）函数
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层

    # 定义模型的前向传播：如何根据输入X返回所需的模型输出 
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)

tensor([[ 0.1053, -0.1218, -0.0074, -0.1148, -0.4139, -0.0383, -0.0866,  0.0313,
         -0.1437, -0.2441],
        [ 0.0115, -0.0548,  0.0589, -0.0830, -0.1385,  0.0562, -0.0472, -0.0564,
         -0.0578, -0.3682]], grad_fn=<AddmmBackward0>)

In [ ]:
# 自定义顺序块
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X